Лабораторная работа №8

**Исследуем датасет MNIST с использованием разных фреймфорков для работы с нейронными сетями**

**Задания к работе:**

Установить Tensor Flow и PyTorch если вы работаете на локальной системе

Часть 1. Распознавание данных MNIST используя многослойный персептрон (MLP)

Используемая конфигурация сети – 3 скрытых слоя по 100 нейронов плюс выходной слой из 10 нейронов. Функции активации выбрать самостоятельно. Реализовать обучение **одинаковой**! конфигурации сети используя Tensor Flow и PyTorch. Вывести метрики классификации. Сравнить время обучения и полученные результаты.


### TensorFlow


In [15]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
from tabulate import tabulate
from time import time

# Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalise data
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert labels into one-hot vectors
y_train = tf.one_hot(y_train, 10, dtype=tf.int32).numpy()
y_test = tf.one_hot(y_test, 10, dtype=tf.int32).numpy()

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
tensorflow_train_time_start = time()
model.fit(x_train, y_train, epochs=10, validation_split=0.1)
tensorflow_train_time_end = time()

# Evaluate the model on test set
tensorflow_evaluation_time_start = time()
loss, accuracy = model.evaluate(x_test, y_test)
tensorflow_evaluation_time_end = time()

print(tabulate([['Training time', tensorflow_train_time_end - tensorflow_train_time_start],
                ['Evaluation time', tensorflow_evaluation_time_end -
                    tensorflow_evaluation_time_start],
                ['Loss', loss],
                ['Accuracy', accuracy]],
               headers=['Metric', 'Value'], tablefmt='github'))

Epoch 1/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.2584 - accuracy: 0.9228 - val_loss: 0.1094 - val_accuracy: 0.9670
Epoch 2/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.1097 - accuracy: 0.9666 - val_loss: 0.0887 - val_accuracy: 0.9733
Epoch 3/10
1688/1688 [==============================] - 4s 3ms/step - loss: 0.0796 - accuracy: 0.9753 - val_loss: 0.0851 - val_accuracy: 0.9757
Epoch 4/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0616 - accuracy: 0.9806 - val_loss: 0.1062 - val_accuracy: 0.9725
Epoch 5/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0500 - accuracy: 0.9842 - val_loss: 0.0867 - val_accuracy: 0.9757
Epoch 6/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0429 - accuracy: 0.9860 - val_loss: 0.0899 - val_accuracy: 0.9777
Epoch 7/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0352 - accuracy: 0.9884 - val_loss: 0.0936 - val_accuracy:

### PyTorch


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

# Load data
train_data = MNIST(root='data', train=True,
                   download=True, transform=ToTensor())
test_data = MNIST(root='data', train=False,
                  download=True, transform=ToTensor())

# Define DataLoader
train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data, batch_size=256)


class MLP(nn.Module):  # Define model
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(28*28, 100)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(100, 100)
        self.relu2 = nn.ReLU()
        self.linear3 = nn.Linear(100, 100)
        self.relu3 = nn.ReLU()
        self.linear4 = nn.Linear(100, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu1(self.linear1(x))
        x = self.relu2(self.linear2(x))
        x = self.relu3(self.linear3(x))
        return self.linear4(x)


model = MLP()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())


# Train the model
pytorch_train_time_start = time()
for epoch in range(10):
    for x, y in train_loader:
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
pytorch_train_time_end = time()

# Evaluate the model on test set
pytorch_evaluation_time_start = time()
with torch.no_grad():
    correct, total = 0, 0
    for x, y in test_loader:
        y_pred = model(x)
        _, predicted = torch.max(y_pred.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()
pytorch_evaluation_time_end = time()

pytorch_test_accuracy = correct / total

print(tabulate([['Training time', pytorch_train_time_end - pytorch_train_time_start],
                ['Evaluation time', pytorch_evaluation_time_end -
                    pytorch_evaluation_time_start],
                ['Accuracy', pytorch_test_accuracy]],
               headers=['Metric', 'Value'], tablefmt='github'))

| Metric          |    Value |
|-----------------|----------|
| Training time   | 92.5752  |
| Evaluation time |  1.39677 |
| Accuracy        |  0.9722  |


In [17]:
print(tabulate([['TensorFlow', tensorflow_train_time_end - tensorflow_train_time_start,
                 tensorflow_evaluation_time_end - tensorflow_evaluation_time_start,
                 accuracy],
                ['PyTorch', pytorch_train_time_end - pytorch_train_time_start,
                 pytorch_evaluation_time_end - pytorch_evaluation_time_start,
                 pytorch_test_accuracy]],
               headers=['Framework', 'Training time',
                        'Evaluation time', 'Accuracy'],
               tablefmt='github'))

| Framework   |   Training time |   Evaluation time |   Accuracy |
|-------------|-----------------|-------------------|------------|
| TensorFlow  |         42.3099 |          0.468867 |     0.9744 |
| PyTorch     |         92.5752 |          1.39677  |     0.9722 |


Часть 2. Распознавание данных MNIST используя сверточную сеть(CNN)

Используемая конфигурация сети – 2 набора слоев свертка+пуллинг (использовать свертку с размером ядра 5 и пулинг с размером 2). Один полносвязный слой на 500 узлов и выходной слой на 10. Остальные параметры выбрать самостоятельно. Внимательно изучите как связываются слои между собой! Реализовать обучение **одинаковой**! конфигурации сети используя Tensor Flow и PyTorch. Вывести метрики классификации. Сравнить время обучения и полученные результаты, а также сравнить с результатами первой части.

Сделать общий вывод. Какой из фреймворков вам понравился больше и почему. Отчет должен содержать 3 файла: 1 – с реализацией на PyTorch, второй - Tensor Flow. Отчет оформить отдельным файлом (третьим)


### PyTorch


In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Define device (GPU or CPU) to be used for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define transform for data augmentation and normalization
transform_train = transforms.Compose([
    transforms.RandomCrop(28, padding=2),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load MNIST dataset
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                      download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                     download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

# Define neural network architecture


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.fc1 = nn.Linear(64 * 4 * 4, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()
net.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

# Train the model for a certain number of epochs
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print the loss after each epoch
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / len(trainloader)))

# Evaluate the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on test set: %d %%' % (100 * correct / total))

[1] loss: 0.499
[2] loss: 0.107
[3] loss: 0.079
[4] loss: 0.063
[5] loss: 0.052
[6] loss: 0.047
[7] loss: 0.042
[8] loss: 0.039
[9] loss: 0.036
[10] loss: 0.035
Accuracy on test set: 99 %


### TensorFlow


In [21]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Reshape input data to three dimensions (height, width, channels)
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

# Define the CNN architecture
model = keras.Sequential(
    [
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu",
                      input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation="softmax"),
    ]
)

# Compile the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam", metrics=["accuracy"])

# Train the model for a certain number of epochs
model.fit(x_train, y_train, batch_size=128, epochs=10, validation_split=0.1)

# Evaluate the model on the test set
score = model.evaluate(x_test, y_test, verbose=0)

print("\n\nTest loss:", score[0])
print("Test accuracy:", score[1])

Epoch 1/10
422/422 [==============================] - 15s 33ms/step - loss: 0.3671 - accuracy: 0.8878 - val_loss: 0.0797 - val_accuracy: 0.9785
Epoch 2/10
422/422 [==============================] - 14s 33ms/step - loss: 0.1113 - accuracy: 0.9653 - val_loss: 0.0595 - val_accuracy: 0.9830
Epoch 3/10
422/422 [==============================] - 14s 33ms/step - loss: 0.0862 - accuracy: 0.9733 - val_loss: 0.0501 - val_accuracy: 0.9865
Epoch 4/10
422/422 [==============================] - 14s 33ms/step - loss: 0.0704 - accuracy: 0.9781 - val_loss: 0.0428 - val_accuracy: 0.9887
Epoch 5/10
422/422 [==============================] - 14s 32ms/step - loss: 0.0626 - accuracy: 0.9808 - val_loss: 0.0385 - val_accuracy: 0.9897
Epoch 6/10
422/422 [==============================] - 14s 32ms/step - loss: 0.0551 - accuracy: 0.9829 - val_loss: 0.0385 - val_accuracy: 0.9907
Epoch 7/10
422/422 [==============================] - 14s 32ms/step - loss: 0.0500 - accuracy: 0.9844 - val_loss: 0.0357 - val_accuracy:

Общий вывод:
TensorFlow понравился больше, так как он интуитивно понятнее и проще в использовании, а также имеет большое количество готовых решений для различных задач, что позволяет сократить время на разработку. Помимо этого, TensorFlow имеет большое количество документации и примеров, что также упрощает работу с ним. Также нельзя не отметить **нативную поддержку GPU**, что позволяет существенно ускорить обучение модели.
